In [3]:
%pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import socket
import sys
from datetime import datetime
import gradio as gr

# --------------------------------------------------
# COPY the relevant classes/functions from client.py
# (Cache, find_in_cache, create_client, connect_to_server, etc.)
# --------------------------------------------------

CLIENT_DIR = 'Client'
cache_list = []
get_history = []

class Cache:
    def __init__(self, filename, content, timestamp):
        self.filename = filename
        self.content = content
        self.timestamp = timestamp
    
    def __str__(self):
        return f"File: {self.filename}\nTimestamp: {self.timestamp}\nContent:\n{self.content}\n"

def find_in_cache(cache_list, filename):
    for cache_entry in cache_list:
        if cache_entry.filename == filename:
            return cache_entry
    return None

def create_client():
    return socket.socket(socket.AF_INET, socket.SOCK_STREAM)

def connect_to_server(client_socket, host='localhost', port=8080):
    client_socket.connect((host, port))

def send_request(client_socket, message, is_binary=False):
    try:
        client_socket.send(message.encode('utf-8'))
        
        if not is_binary:
            response = client_socket.recv(4096).decode('utf-8')
            return response

        # Binary reading
        response = b""
        content_length = None
        header_end = None

        while True:
            chunk = client_socket.recv(4096)
            if not chunk:
                break
            response += chunk

            if header_end is None:
                header_end = response.find(b"\r\n\r\n")
                if header_end != -1:
                    header_data = response[:header_end].decode('utf-8', errors='replace')
                    lines = header_data.split("\r\n")
                    for line in lines:
                        if line.lower().startswith("content-length:"):
                            parts = line.split(":", 1)
                            content_length = int(parts[1].strip())
                            break

            if header_end is not None and content_length is not None:
                body_len = len(response) - (header_end + 4)
                if body_len >= content_length:
                    break
        
        return response
    except Exception as e:
        return f"Error sending/receiving data: {e}"

def handle_binary_file(response):
    try:
        header_end = response.find(b'\r\n\r\n')
        if header_end == -1:
            print("ERROR: Could not find end of headers")
            return None
        
        headers = response[:header_end].decode('utf-8', errors='ignore')
        content = response[header_end + 4:]  # Skip \r\n\r\n

        if '200 OK' not in headers:
            print(f"ERROR: Response did not return 200 OK:\n{headers}")
            return None

        # Check Content-Type
        content_type = None
        content_length = None
        for line in headers.split('\r\n'):
            if line.lower().startswith('content-type:'):
                content_type = line.split(': ')[1].strip()
            elif line.lower().startswith('content-length:'):
                content_length = int(line.split(': ')[1].strip())

        # Print Content-Type and Length
        print(f"DEBUG: Content-Type: {content_type}, Content-Length: {content_length}")
        print(f"DEBUG: Extracted content length: {len(content)}")

        if content_length and len(content) < content_length:
            print("ERROR: Incomplete file received!")
            return None  # Prevent using an incomplete file

        return content  # Return binary data

    except Exception as e:
        print(f"ERROR: Exception while handling binary file: {e}")
        return None


# --------------------------------------------------
# MAIN FUNCTION that handles the request using Gradio
# --------------------------------------------------

def client_request(method, filename, data, host, port):
    """
    Receives the Gradio UI input,
    performs the HTTP request using our client logic,
    and returns a string with the result/response.
    """

    # Basic validations
    if not method:
        return "Please specify a method (GET, POST, PUT, DELETE)."
    method = method.upper()
    if method not in ["GET","POST","PUT","DELETE"]:
        return f"Invalid method: {method}"
    
    result_messages = []

    # Create and connect socket
    client = create_client()
    try:
        connect_to_server(client, host, int(port))
    except Exception as e:
        return f"Error connecting to server {host}:{port} => {e}"
    
    if method == "GET":
        path = f"/{filename}"
        # Decide text vs. binary by extension
        ext = filename.split('.')[-1].lower()
        if ext in ['png','jpg','jpeg','gif','svg','webp','mp3','wav','ogg','mp4','avi']:
            is_binary = True
        else:
            is_binary = False

        request = f"GET {path} HTTP/1.1\r\n"
        request += f"Host: {host}\r\n\r\n"

        response = send_request(client, request, is_binary=is_binary)
        client.close()

        if is_binary:
            if isinstance(response, bytes):
                content = handle_binary_file(response)
                print(content[:100]) 
                if content:
                    # Save the binary file to Client/filename
                    save_path = os.path.join(CLIENT_DIR, filename)
                    with open(save_path, 'wb') as f:
                        f.write(content)
                    result_messages.append(f"Binary file saved to: {save_path}")
                else:
                    result_messages.append("Failed to process binary content.")
            else:
                # Probably an error string
                result_messages.append(str(response))
        else:
            # Treat as text
            if not isinstance(response, str):
                response = response.decode('utf-8', errors='replace') if isinstance(response, bytes) else str(response)
            result_messages.append(response)

    elif method == "PUT":
        if not filename:
            return "You must specify a filename for PUT."

        if not data:
            data = "empty content"

        path = f"/{filename}"
        request = f"PUT {path} HTTP/1.1\r\n"
        request += f"Host: {host}\r\n"
        request += "Content-Type: text/plain\r\n"
        request += f"Content-Length: {len(data)}\r\n"
        request += "\r\n"
        request += data

        response = send_request(client, request, is_binary=False)
        client.close()
        if response:
            result_messages.append(response)
        else:
            result_messages.append("No response from server.")

    elif method == "POST":
        path = f"/{filename}"
        if not data:
            data = "{}"
        request = f"POST {path} HTTP/1.1\r\n"
        request += f"Host: {host}\r\n"
        request += "Content-Type: application/json\r\n"
        request += f"Content-Length: {len(data)}\r\n"
        request += "\r\n"
        request += data

        response = send_request(client, request, is_binary=False)
        client.close()
        if response:
            result_messages.append(response)
        else:
            result_messages.append("No response from server.")

    elif method == "DELETE":
        path = f"/{filename}"
        request = f"DELETE {path} HTTP/1.1\r\n"
        request += f"Host: {host}\r\n\r\n"

        response = send_request(client, request, is_binary=False)
        client.close()
        if response:
            result_messages.append(response)
        else:
            result_messages.append("No response from server.")

    return "\n".join(result_messages)


# --------------------------------------------------
# GRADIO UI
# --------------------------------------------------
with gr.Blocks() as demo:
    gr.Markdown("## HTTP Client")

    host = gr.Textbox(value="localhost", label="Host")
    port = gr.Number(value=8080, label="Port", precision=0)
    
    method = gr.Radio(choices=["GET","POST","PUT","DELETE"], value="GET", label="HTTP Method")
    filename = gr.Textbox(label="Filename (e.g., index.html, image.png, etc.)")
    data = gr.Textbox(label="Request Body (for POST/PUT)")

    output = gr.Textbox(label="Server Response", lines=10)

    run_button = gr.Button("Send Request")

    def on_click_run(method, filename, data, host, port):
        return client_request(method, filename, data, host, port)

    run_button.click(
        fn=on_click_run,
        inputs=[method, filename, data, host, port],
        outputs=[output]
    )

if __name__ == "__main__":
    demo.launch(debug=True)


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
